# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from typing import Sequence, cast

# analytic imports
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import readabs as ra
from readabs import metacol as mc
import mgplot as mg

# local imports
from abs_helper import ANNUAL_CPI_TARGET_RANGE
from abs_structured_capture import ReqsDict, ReqsTuple, get_abs_data

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
CHART_DIR = "./CHARTS/Inflation/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()

# some plotting constants
LFOOTER = "Australia. Orig = Original series. SA = Seasonally adjusted series. "
YEARLY_RANGE = {"axhspan": ANNUAL_CPI_TARGET_RANGE}

# display charts in this notebook
SHOW = False

## Plotting

### Catify

In [3]:
def catify(w: ReqsDict) -> str:
    """Get a string of ABS catalogues from a ReqsDict."""

    cats = set()
    for key in w.keys():
        cats.add(w[key].cat if w[key].cat else w[key].table)
    return f"ABS: {', '.join(sorted(cats))}"

### Monthify

In [4]:
def monthify(d: dict[str, Series]) -> dict[str, Series]:
    """Convert a Series with Quarterly PeriodIndex to a monthly Series."""

    package = {}
    for label, s in d.items():
        match cast(pd.PeriodIndex, s.index).freqstr[0]:
            case "M":
                package[label] = s
            case "Q":
                package[label] = ra.qtly_to_monthly(s)
            case other:
                raise ValueError(f"Unexpected frequency '{other}' in series '{label}'")
    return package

### Ten contrasty colours 

In [5]:
COLOURS = [
    # 10 relatively distinct colours
    'tab:blue',
    'tab:orange', 
    'tab:green',
    'tab:red',
    'tab:purple',
    'tab:brown',
    'tab:pink',
    'tab:gray',
    'tab:olive',
    'tab:cyan',
]

### Chart the key CPI measures

In [6]:
OLD_CPI = "./ABS-Data/Qrtly-CPI-Time-series-spreadsheets-all.zip"

W_CPI_ONLY_MEASURES: ReqsDict = {
    # - Reminder: cat, table, did, stype, unit, seek_yr_growth, calc_growth, zip_file
    # --- the next four will need to be updated when new Qrtly CPI data is available ---
    "Qrtly Headline CPI (Orig)" :
        ReqsTuple("", "640106", "All groups CPI ;", "O", "", True, False, OLD_CPI),
    "Qrtly Headline CPI (SA)" :
        ReqsTuple("", "640106", "All groups CPI, seasonally adjusted", "S", "", True, False, OLD_CPI),
    "Qrtly Trimmed Mean CPI (SA)" :
        ReqsTuple("", "640106", "Trimmed Mean", "S", "", True, False, OLD_CPI),
    "Qrtly Weighted Median CPI (SA)" :
        ReqsTuple("", "640106", "Weighted Median", "S", "", True, False, OLD_CPI),
   # --- the next four are the new monthly CPI measures ---
    "Monthly Headline CPI (Orig)" :
        ReqsTuple("6401.0", "640106", "All groups CPI ;", "O", "", True, False, ""),
    "Monthly Headline CPI (SA)" :
        ReqsTuple("6401.0", "640106", "All groups CPI, seasonally adjusted", "S", "", True, False, ""),
    "Monthly Trimmed Mean CPI (SA)" :
        ReqsTuple("6401.0", "640106", "Trimmed Mean", "S", "", True, False, ""),
    "Monthly Weighted Median CPI (SA)" :
        ReqsTuple("6401.0", "640106", "Weighted Median", "S", "", True, False, ""),
}

In [7]:
def cpi_multi_chart():
    """Plot multi-measure CPI chart."""

    # this will need to be updated when new quarterly data is available
    combined = get_abs_data(W_CPI_ONLY_MEASURES)
    combined = monthify(combined)

    # convert to a dataframe and plot
    frame = pd.DataFrame(combined)
    mg.multi_start(
        starts=[0,-19],
        function=mg.line_plot_finalise,
        data=frame,
        title="Australian Consumer Price Index (CPI) Measures",
        ylabel="CPI (annual % change)",
        **YEARLY_RANGE,
        legend={"loc": "best", "ncol": 2, "fontsize": "x-small"},
        lfooter=LFOOTER,
        color=COLOURS,
        rfooter=catify(W_CPI_ONLY_MEASURES),
        y0=True,
        show=SHOW
    )


cpi_multi_chart()

### Chart a selection of ABS inflation measures

In [8]:
W_MULTI_MEASURE = {
    # --- the first three will need to be updated when new Qrtly CPI data is available ---
    "Qrtly Headline CPI (SA)" :
        ReqsTuple("", "640106", "All groups CPI, seasonally adjusted", "S", "", True, False, OLD_CPI),
    "Qrtly Trimmed Mean CPI (SA)" :
        ReqsTuple("", "640106", "Trimmed Mean", "S", "", True, False, OLD_CPI),
    "Qrtly Weighted Median CPI (SA)" :
        ReqsTuple("", "640106", "Weighted Median", "S", "", True, False, OLD_CPI),
    # --- the remaining measures should be okay ---
    "Monthly Headline CPI (SA)" :
        ReqsTuple("6401.0", "640106", "All groups CPI, seasonally adjusted", "S", "", True, False, ""),
    "Monthly Trimmed Mean CPI (SA)" :
        ReqsTuple("6401.0", "640106", "Trimmed Mean", "S", "", True, False, ""),
    "Monthly Weighted Median CPI (SA)" :
        ReqsTuple("6401.0", "640106", "Weighted Median", "S", "", True, False, ""),
    "Producer Price Index (Orig)": 
        ReqsTuple("6427.0", "642701", "Final ;  Total (Source) ;", "O", "", True, False, ""),
    "Wage Price Index (All sectors) (SA)": 
        ReqsTuple(
            "6345.0", 
            "634501", 
            "Total hourly rates of pay excluding bonuses ;  Private and Public ;  All industries ;", 
            "S", 
            "",
            True, 
            False, 
            ""
        ),
    "Households implicit price deflator (SA)": 
        ReqsTuple(
            "5206.0", 
            "5206005_Expenditure_Implicit_Price_Deflators",
            "Households ;  Final consumption expenditure ;",
            "S",
            "Index",
            False,
            True,
            ""
        ), 
    "GNE implicit price deflator (SA)": 
        ReqsTuple(
            "5206.0", 
            "5206005_Expenditure_Implicit_Price_Deflators",
            "Gross national expenditure ;",
            "S",
            "Index",
            False,
            True,
            ""
        ), 
}

In [9]:
def multi_measure_plot() -> None:
    combined = get_abs_data(W_MULTI_MEASURE)
    combined = monthify(combined)

    # Plot
    frame = pd.DataFrame(combined)
    mg.multi_start(
        starts=[pd.Period("1989-12"), pd.Period("2019-12"), pd.Period("2022-12"), pd.Period("2024-06")],
        function=mg.line_plot_finalise,
        data=frame,
        title="Australian Inflation Measures",
        ylabel="Inflation (annual % change)",
        **YEARLY_RANGE,
        legend={"loc": "best", "ncol": 3, "fontsize": "xx-small"},
        lfooter=LFOOTER,
        rfooter=catify(W_MULTI_MEASURE),
        color=COLOURS,
        y0=True,
        show=SHOW
    )


multi_measure_plot()

### Individual series growth charts

In [10]:
W_GROWTH_CHARTS: ReqsDict = {  # used for individual growth plots
    # --- Quarterly CPI measures (from old zip file) ---
    "Qrtly Headline CPI (SA)":
        ReqsTuple("", "640106", "All groups CPI, seasonally adjusted", "S", "Index Numbers", False, False, OLD_CPI),
    "Qrtly Trimmed Mean CPI (SA)":
        ReqsTuple("", "640106", "Trimmed Mean", "S", "Index Numbers", False, False, OLD_CPI),
    "Qrtly Weighted Median CPI (SA)":
        ReqsTuple("", "640106", "Weighted Median", "S", "Index Numbers", False, False, OLD_CPI),
    "Qrtly CPI Goods (Orig)":
        ReqsTuple("", "640106", "All groups, goods component", "O", "Index Numbers", False, False, OLD_CPI),
    "Qrtly CPI Services (Orig)":
        ReqsTuple("", "640106", "All groups, services component", "O", "Index Numbers", False, False, OLD_CPI),
    "Qrtly CPI Tradables (Orig)":
        ReqsTuple("", "640106", "Tradables ;", "O", "Index Numbers", False, False, OLD_CPI),
    "Qrtly CPI Non-tradables (Orig)":
        ReqsTuple("", "640106", "Non-tradables ;", "O", "Index Numbers", False, False, OLD_CPI),
    # --- Monthly CPI measures ---
    "Monthly Headline CPI (SA)":
        ReqsTuple("6401.0", "640106", "All groups CPI, seasonally adjusted", "S", "Index Numbers", False, False, ""),
    "Monthly Trimmed Mean CPI (SA)":
        ReqsTuple("6401.0", "640106", "Trimmed Mean", "S", "Index Numbers", False, False, ""),
    "Monthly Weighted Median CPI (SA)":
        ReqsTuple("6401.0", "640106", "Weighted Median", "S", "Index Numbers", False, False, ""),
    "Monthly CPI Goods (Orig)":
        ReqsTuple("6401.0", "640106", "All groups, goods component", "O", "Index Numbers", False, False, ""),
    "Monthly CPI Services (Orig)":
        ReqsTuple("6401.0", "640106", "All groups, services component", "O", "Index Numbers", False, False, ""),
    "Monthly CPI Tradables (Orig)":
        ReqsTuple("6401.0", "640106", "Tradables ;", "O", "Index Numbers", False, False, ""),
    "Monthly CPI Non-tradables (Orig)":
        ReqsTuple("6401.0", "640106", "Non-tradables ;", "O", "Index Numbers", False, False, ""),
    # --- Other inflation measures ---
    "Producer Price Index (Orig)":
        ReqsTuple("6427.0", "642701", "Final ;  Total (Source) ;", "O", "Index Numbers", False, False, ""),
    "Wage Price Index (All sectors) (SA)":
        ReqsTuple("6345.0", "634501",
            "Quarterly Index ;  Total hourly rates of pay excluding bonuses ;  Australia ;  Private and Public",
            "S", "Index Numbers", False, False, ""),
    # --- Implicit price deflators use "Index" ---
    "Households implicit price deflator (SA)":
        ReqsTuple("5206.0", "5206005_Expenditure_Implicit_Price_Deflators",
            "Households ;  Final consumption expenditure ;", "S", "Index", False, False, ""),
    "GNE implicit price deflator (SA)":
        ReqsTuple("5206.0", "5206005_Expenditure_Implicit_Price_Deflators",
            "Gross national expenditure ;", "S", "Index", False, False, ""),
}

In [11]:
def growth_charts() -> None:
    """Plot growth charts for various ABS inflation measures."""    

    data = get_abs_data(W_GROWTH_CHARTS)
    
    for key, series in data.items():
        frame = mg.calc_growth(series)
        mg.growth_plot_finalise(
            data=frame,
            plot_from=-19,
            title=f"Growth: {key}",
            ylabel="Per cent",
            lfooter=LFOOTER,
            rfooter=f"ABS {W_GROWTH_CHARTS[key].cat if W_GROWTH_CHARTS[key].cat else W_GROWTH_CHARTS[key].table}",
            show=SHOW,
        )


growth_charts()

### Foursomes (Two Series by Monthly and Quarterly)

In [12]:
# --- Foursome 1: Goods vs Services ---
W_GOODS_SERVICES: ReqsDict = {
    "Qrtly CPI Goods (Orig)":
        ReqsTuple("", "640106", "All groups, goods component", "O", "", True, False, OLD_CPI),
    "Qrtly CPI Services (Orig)":
        ReqsTuple("", "640106", "All groups, services component", "O", "", True, False, OLD_CPI),
    "Monthly CPI Goods (Orig)":
        ReqsTuple("6401.0", "640106", "All groups, goods component", "O", "", True, False, ""),
    "Monthly CPI Services (Orig)":
        ReqsTuple("6401.0", "640106", "All groups, services component", "O", "", True, False, ""),
}

# --- Foursome 2: Tradables vs Non-tradables ---
W_TRADABLES: ReqsDict = {
    "Qrtly CPI Tradables (Orig)":
        ReqsTuple("", "640106", "Tradables ;", "O", "", True, False, OLD_CPI),
    "Qrtly CPI Non-tradables (Orig)":
        ReqsTuple("", "640106", "Non-tradables ;", "O", "", True, False, OLD_CPI),
    "Monthly CPI Tradables (Orig)":
        ReqsTuple("6401.0", "640106", "Tradables ;", "O", "", True, False, ""),
    "Monthly CPI Non-tradables (Orig)":
        ReqsTuple("6401.0", "640106", "Non-tradables ;", "O", "", True, False, ""),
}

# --- Foursome 3: Trimmed Mean vs Weighted Median (SA available) ---
W_CORE_CPI: ReqsDict = {
    "Qrtly Trimmed Mean CPI (SA)":
        ReqsTuple("", "640106", "Trimmed Mean", "S", "", True, False, OLD_CPI),
    "Qrtly Weighted Median CPI (SA)":
        ReqsTuple("", "640106", "Weighted Median", "S", "", True, False, OLD_CPI),
    "Monthly Trimmed Mean CPI (SA)":
        ReqsTuple("6401.0", "640106", "Trimmed Mean", "S", "", True, False, ""),
    "Monthly Weighted Median CPI (SA)":
        ReqsTuple("6401.0", "640106", "Weighted Median", "S", "", True, False, ""),
}

W_FOURSOMES: dict[str, ReqsDict] = {
    "Goods vs Services": W_GOODS_SERVICES,
    "Tradables vs Non-tradables": W_TRADABLES,
    "Trimmed Mean vs Weighted Median": W_CORE_CPI,
}

In [13]:
def foursome_charts() -> None:
    """Plot the various CPI measure foursomes."""
    colors = ("darkblue", "darkorange", "darkblue", "darkorange")
    styles = ("-", "-", "--", "--")
    months_in_year = 12
    for title, reqs in W_FOURSOMES.items():
        combined = get_abs_data(reqs)
        combined = monthify(combined)
        frame = pd.DataFrame(combined)    
        mg.multi_start(
            starts=[0,-(8*months_in_year+1), -(2*months_in_year+1)],
            function=mg.line_plot_finalise,
            data=frame,
            title=f"Australian Inflation Measures: {title}",
            ylabel="CPI (annual % change)",
            **YEARLY_RANGE,
            legend={"loc": "best", "ncol": 2, "fontsize": "x-small"},
            lfooter=LFOOTER,
            color=colors,
            style=styles,
            rfooter=catify(reqs),
            y0=True,
            show=SHOW
    )


foursome_charts()

### Phillips Curves

In [14]:
W_PHILLIPS = {
    # --- the first one will need to be updated when new Qrtly CPI data is available ---
    "Qrtly Trimmed Mean CPI (SA)" :
        ReqsTuple("", "640106", "Trimmed Mean", "S", "", True, False, OLD_CPI),
    # --- the remaining measures should be okay ---
    "Households implicit price deflator (SA)": 
        ReqsTuple(
            "5206.0", 
            "5206005_Expenditure_Implicit_Price_Deflators",
            "Households ;  Final consumption expenditure ;",
            "S",
            "Index",
            False,
            True,
            ""
        ), 
    "Unemployment rate monthly (SA)":
        ReqsTuple("6202.0", "6202001", "Unemployment rate ;  Persons ;", "S", "", False, False, ""),
}


In [15]:
def xy_plot(
    frame: DataFrame,
    line_label: str,
    point_labels: Sequence[int] | None = None,
    ax = None,
    **kwargs,
) -> Axes:
    """ "Plot and label the heart of the curve.

    Arguments:
    frame is a two column DataFrame, first col is x vales, 2nd col is y values.
    line_label is the label for the line.
    point_labels is the points to label.

    Returns:
    An Axes object."""

    width = kwargs.pop("width", 2)
    color = kwargs.pop("color", "royalblue")
    
    ax = frame.plot(
        x=frame.columns[0],
        y=frame.columns[1],
        lw=width,
        color=color,
        label=line_label if line_label else "_nolegend_",
        ax=ax,
    )
    point_labels = [] if point_labels is None else point_labels
    for n in point_labels:
        # Label the start and end
        ax.text(
            frame[frame.columns[0]].iloc[n],
            frame[frame.columns[1]].iloc[n],
            f"{frame.index[n]} ",
            fontsize="x-small",
            ha="right",
        )
    return ax

In [16]:
def add_regression(
    ax,
    frame: DataFrame,
    line_label: str,
    **kwargs: int | str,
) -> None:
    """Fit a polynomial regression line to the data."""

    color: str = str(kwargs.get("color", "darkred"))
    degree: int = int(kwargs.get("degree", 1))
    linestyle: str = str(kwargs.get("linestyle", "--"))
    width: float = float(kwargs.get("width", 0.75))

    model = np.poly1d(
        np.polyfit(frame[frame.columns[0]], frame[frame.columns[1]], degree)
    )
    polyline = np.linspace(
        frame[frame.columns[0]].min(), frame[frame.columns[0]].max(), 50
    )
    ax.plot(
        polyline,
        model(polyline),
        color=color,
        linestyle=linestyle,
        lw=width,
        label=line_label if line_label else "_nolegend_",
    )

In [17]:
def phillips_curve() -> None:
    """Produce a Phillips Curve chart."""

    # --- Organise the data
    data = get_abs_data(W_PHILLIPS)
    ue_rate = ra.monthly_to_qtly(data["Unemployment rate monthly (SA)"])
    hheipd = data["Households implicit price deflator (SA)"]
    tm_cpi = data["Qrtly Trimmed Mean CPI (SA)"]

    inflation = {
        "Trimmed Mean CPI": tm_cpi,
        "Household Expenditure IPD": hheipd,
    }

    start = "2021Q1"  # "2009Q4"  #
    for label, series in inflation.items():

        frame = pd.DataFrame({label: series, "_Unemployment Rate": ue_rate})
        last: float = (
            cast(float, frame.iloc[-1, 1]) if frame.iloc[-1].isna().any() else 0.0
        )
        last_date = frame.index[-1]
        frame = frame.loc[
            lambda x: x.index >= start
        ].dropna()  # drop the last row if it has a NaN

        # --- Plot the data
        ax = xy_plot(frame, "Phillips curve", point_labels=(0, -1))
        add_regression(ax, frame, "Stylised Phillips curve", degree=3)

        ax.axvline(
            2.5, color="darkblue", linestyle=":", lw=0.75, label="2.5% Inflation target"
        )
        if last > 0.0:
            ax.axhline(
                last,
                color="darkgreen",
                linestyle="-.",
                lw=0.75,
                label=f"Unemployment rate {last_date}",
            )
        mg.finalise_plot(
            ax,
            title=f"Phillips Curve: {label} vs Unemployment",
            ylabel="Unemployment Rate (%)",
            xlabel=f"{label} Annual Growth Rate (%)",
            lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean. ",
            rfooter=catify(W_PHILLIPS),
            legend=True,
            show=SHOW,
        )


phillips_curve()

In [18]:
def stylised_phillips():
    """Produce a Stylised Phillips Curve chart for different periods."""
    
    # --- Organise the data
    data = get_abs_data(W_PHILLIPS)
    ue_rate = ra.monthly_to_qtly(data["Unemployment rate monthly (SA)"])
    tm_cpi = data["Qrtly Trimmed Mean CPI (SA)"]

    # --- Define the periods to plot
    period_sets = [
        [
            list(pd.period_range(pd.Period("2021Q1", freq="Q"), tm_cpi.index[-1])),
            "brown",
            3,
        ],
        [
            list(pd.period_range(pd.Period("2010Q1", freq="Q"), pd.Period("2019Q4", freq="Q"))),
            "blue",
            1,
        ],
        [
            list(pd.period_range(pd.Period("2003Q1", freq="Q"), pd.Period("2008Q4", freq="Q"))),
            "orange",
            3,
        ],
        [
            list(pd.period_range(pd.Period("1998Q1", freq="Q"), pd.Period("2002Q4", freq="Q"))),
            "skyblue",
            1,
        ],
    ]
    
    _, ax = plt.subplots()
    
    for start_stop, color, degree in period_sets:
        assert len(start_stop) >= 2, f"start_stop len={len(start_stop)} must have start and end"
        label = f"{start_stop[0]}-{start_stop[-1]}"
        
        frame = pd.DataFrame({
            "Trimmed Mean CPI": tm_cpi,
            "Unemployment Rate": ue_rate,
        })
        frame = frame.loc[start_stop[0]:start_stop[-1]].dropna()

        xy_plot(
            frame,
            line_label="",
            point_labels=[],
            ax=ax,
            width=0.5,
            color=color,
        )
        
        add_regression(
            ax,
            frame,
            label,
            degree=degree,
            color=color,
            linestyle="-",
            width=3,
        )

    axvline = {
        "x": 2.5,
        "color": "darkblue",
        "linestyle": ":",
        "lw": 0.75,
        "label": "2.5% Inflation target",
    }
    
    mg.finalise_plot(
        ax,
        title="Stylised Phillips Curves: Trimmed Mean CPI vs Unemployment",
        ylabel="Unemployment Rate (%)",
        xlabel="Trimmed Mean CPI Annual Growth Rate (%)",
        lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean.",
        rfooter=catify(W_PHILLIPS),
        axvline=axvline,
        legend=True,
        show=SHOW,
    )


stylised_phillips()

## Beveridge curve

In [19]:
W_BEVERIDGE: ReqsDict = {
    "Unemployment rate monthly (SA)":
        ReqsTuple("6202.0", "6202001", "Unemployment rate ;  Persons ;", "S", "", False, False, ""),
    "Labour force (SA)":
        ReqsTuple("6202.0", "6202001", "Labour force total ;  Persons ;", "S", "", False, False, ""),
    "Job vacancies (SA)":
        ReqsTuple("6354.0", "6354001", "Job Vacancies ;  Australia ;", "S", "", False, False, ""),
}

In [20]:
def beveridge_curve() -> None:
    """Plot a Beveridge curve."""

    # --- Organise the data --- we need to shift the unemployment and labour force data by 1 month
    data = get_abs_data(W_BEVERIDGE)
    ue_rate = ra.monthly_to_qtly(data["Unemployment rate monthly (SA)"]. shift(1))  # Q1 = Dec, Jan, Feb / called Q-Dec
    lf = ra.monthly_to_qtly(data["Labour force (SA)"].shift(1)) # Q1 = Dec, Jan, Feb / but called Q-Dec
    jv = data["Job vacancies (SA)"]
    jv.index = pd.PeriodIndex(jv.index.astype(str), freq='Q-DEC')  # relabel index so that Q-Nov -> Q-Dec
    jvr = jv / lf * 100

    frame = pd.DataFrame({"Unemployment Rate": ue_rate, "Job Vacancy Rate": jvr})
    start = "2010Q2"
    frame = frame.loc[lambda x: x.index >= start].dropna()

    # --- Plot the next chart
    labels = [k for k in range(-1, -len(frame), -4) if k > -len(frame)]
    ax = xy_plot(frame, "Beveridge curve", point_labels=labels)
    points = frame.loc[lambda x: x.index < "2020Q2"].index.union(
        frame.loc[lambda x: x.index >= "2023Q3"].index
    )
    f = frame.loc[points]
    add_regression(ax, f, "Pre-COVID Beveridge curve (stylised)", degree=2)
    points = frame.loc[lambda x: (x.index >= "2020Q3") & (x.index <= "2022Q2")].index
    f = frame.loc[points]
    add_regression(
        ax,
        f,
        "Immediate post-COVID Beveridge curve (stylised)",
        degree=2,
        color="darkgreen",
        linestyle="-.",
    )
    mg.finalise_plot(
        ax,
        title="Beveridge Curve: Unemployment Rate vs Job Vacancy Rate",
        ylabel="Job Vacancy Rate (%)",
        xlabel="Unemployment Rate (%)",
        lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean "
        + "(quarters ending Feb, May, Aug, Nov).",
        rfooter=catify(W_BEVERIDGE),
        legend=True,
        show=SHOW,
    )


beveridge_curve()

## Finished

In [21]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-12-12 11:43:04

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.8.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

readabs   : 0.1.8
matplotlib: 3.10.7
pandas    : 2.3.3
typing    : 3.10.0.0
numpy     : 2.3.5
mgplot    : 0.2.14

Watermark: 2.5.0



In [22]:
print("End of Inflation Analysis")

End of Inflation Analysis
